<a href="https://colab.research.google.com/github/KoreTane/KoreTane/blob/main/EDAProfitForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install dash

In [2]:
import pandas as pd  # Manipulação de dados (DataFrames).
import numpy as np  # Operações numéricas e arrays.
import matplotlib.pyplot as plt  # Gráficos estáticos.
import seaborn as sns  # Gráficos estatísticos.
import plotly.graph_objects as go  # Gráficos interativos.
from plotly.subplots import make_subplots  # Subgráficos no Plotly.
from dash import Dash, dcc, html, Input, Output  # Aplicações web interativas.
from datetime import datetime  # Manipulação de datas.
from sklearn.model_selection import train_test_split  # Divisão de dados em treino/teste.
from sklearn.ensemble import RandomForestRegressor  # Modelo de regressão Random Forest.
from sklearn.metrics import mean_absolute_error, mean_squared_error  # Métricas de erro.

In [3]:
# Ler a aba "Receita"
df_receita = pd.read_excel('fluxo_caixa_cbs.xlsx', sheet_name='Receita')

# Ler a aba "Despesa"
df_despesa = pd.read_excel('fluxo_caixa_cbs.xlsx', sheet_name='Despesa')

In [4]:
df_despesa

,Data,Descrição,Valor,Tipo
0,2023-01-03,Sabesp,-59.46,Retirada pessoal
1,2023-01-05,EDP,-226.15,Despesas
2,2023-01-05,Sabesp,-119.15,Despesas
3,2023-01-09,EDP,-97.71,Retirada pessoal
4,2023-01-12,Internet,-140.12,Despesas
...,...,...,...,...
489,2024-04-23,Instalação bebedouro,-209.85,Reforma/Manutenção
490,2024-04-24,Suprimentos,-21.29,Despesas
491,2024-04-24,Suprimentos,-17.44,Despesas
492,2024-04-24,Tomada bebedouro,-26.90,Reforma/Manutenção


In [5]:
df_receita

,Data,Descrição,Valor,Tipo
0,2023-01-11,#CLIENTE#23017,244.02,Debito
1,2023-01-12,#CLIENTE#23018,199.00,Dinheiro
2,2023-01-13,#CLIENTE#23002,237.12,Crédito
3,2023-01-13,#CLIENTE#23003,237.12,Crédito
4,2023-01-14,#CLIENTE#23007,250.00,Dinheiro
...,...,...,...,...
1306,2024-04-15,#CLIENTE#24024,230.00,Dinheiro
1307,2024-04-17,#CLIENTE#24024,164.07,Crédito
1308,2024-04-23,#CLIENTE#24023,316.89,Crédito
1309,2024-04-23,#CLIENTE#24023,358.34,Crédito


In [6]:
# Agrupando por 'Tipo' e somando os 'Valor'
result_receita = df_receita.groupby('Tipo')['Valor'].sum().reset_index()
result_despesa = df_despesa.groupby('Tipo')['Valor'].sum().reset_index()

In [7]:
result_despesa
result_receita

,Tipo,Valor
0,Aporte pessoal,522.13
1,Boleto,38017.52
2,Crédito,301170.75
3,Debito,6443.54
4,Dinheiro,105575.28
5,Espetáculo,28161.19
6,Investimento,81.62


In [8]:
result_despesa

,Tipo,Valor
0,Despesas,-161024.25
1,Débito,-4076.38
2,Empréstimo,-54580.25
3,Impostos,-27555.00
4,Pró-labore,-22673.46
5,Reforma/Manutenção,-9738.55
6,Retirada pessoal,-44644.30
7,Salários,-176072.41
8,Tarifas Bancárias,-1312.64


In [9]:
print(result_despesa)
print(result_receita)

                 Tipo      Valor
0            Despesas -161024.25
1              Débito   -4076.38
2          Empréstimo  -54580.25
3            Impostos  -27555.00
4          Pró-labore  -22673.46
5  Reforma/Manutenção   -9738.55
6    Retirada pessoal  -44644.30
7            Salários -176072.41
8   Tarifas Bancárias   -1312.64
             Tipo      Valor
0  Aporte pessoal     522.13
1          Boleto   38017.52
2         Crédito  301170.75
3          Debito    6443.54
4        Dinheiro  105575.28
5      Espetáculo   28161.19
6    Investimento      81.62


In [10]:
despesa_agrupada = result_despesa.groupby('Tipo')['Valor'].sum().reset_index()

# Renomear a coluna para maior clareza
despesa_agrupada.columns = ['Descrição', 'Total']

# Exibir a matriz agrupada
print("Matriz de Despesa Agrupada:")
print(despesa_agrupada)

Matriz de Despesa Agrupada:
            Descrição      Total
0            Despesas -161024.25
1              Débito   -4076.38
2          Empréstimo  -54580.25
3            Impostos  -27555.00
4          Pró-labore  -22673.46
5  Reforma/Manutenção   -9738.55
6    Retirada pessoal  -44644.30
7            Salários -176072.41
8   Tarifas Bancárias   -1312.64


In [11]:
receita_agrupada = result_receita.groupby('Tipo')['Valor'].sum().reset_index()

# Renomear a coluna para maior clareza
receita_agrupada.columns = ['Tipo', 'ValorTotal']

# Exibir a matriz agrupada
print("Matriz de Receita Agrupada:")
print(receita_agrupada)

Matriz de Receita Agrupada:
             Tipo  ValorTotal
0  Aporte pessoal      522.13
1          Boleto    38017.52
2         Crédito   301170.75
3          Debito     6443.54
4        Dinheiro   105575.28
5      Espetáculo    28161.19
6    Investimento       81.62


In [12]:
# Agrupar receitas e despesas por Data
receita_agrupada = df_receita.groupby('Data')['Valor'].sum().reset_index()
receita_agrupada.rename(columns={'Valor': 'Receita'}, inplace=True)

despesa_agrupada = df_despesa.groupby('Data')['Valor'].sum().reset_index()
despesa_agrupada.rename(columns={'Valor': 'Despesa'}, inplace=True)

# Tornar as despesas negativas
despesa_agrupada['Despesa'] = -despesa_agrupada['Despesa']  # Tornar as despesas como valores negativos

# Unir os dois DataFrames
lucro_df = pd.merge(receita_agrupada, despesa_agrupada, on='Data', how='outer').fillna(0)

# Calcular o lucro corretamente usando as colunas do DataFrame lucro_df
lucro_df['Lucro'] = lucro_df['Receita'] + lucro_df['Despesa']  # Lucro = Receita - Despesa (que é negativa)

# Exibir o DataFrame de lucro
print(lucro_df)


          Data  Receita  Despesa    Lucro
0   2023-01-11   244.02     0.00   244.02
1   2023-01-12   199.00   140.12   339.12
2   2023-01-13   474.24     0.00   474.24
3   2023-01-14   250.00  7885.62  8135.62
4   2023-01-16   374.95   131.98   506.93
..         ...      ...      ...      ...
310 2024-04-11     0.00   197.50   197.50
311 2024-04-12     0.00   180.34   180.34
312 2024-04-18     0.00   913.10   913.10
313 2024-04-20     0.00  4800.11  4800.11
314 2024-04-30     0.00   427.98   427.98

[315 rows x 4 columns]


In [13]:
# Converter a coluna 'Data' para datetime
df_receita['Data'] = pd.to_datetime(df_receita['Data'])
df_despesa['Data'] = pd.to_datetime(df_despesa['Data'])

# Agrupar receitas e despesas por mês
receita_agrupada = df_receita.resample('M', on='Data')['Valor'].sum().reset_index()
receita_agrupada.rename(columns={'Valor': 'Receita'}, inplace=True)

despesa_agrupada = df_despesa.resample('M', on='Data')['Valor'].sum().reset_index()
despesa_agrupada.rename(columns={'Valor': 'Despesa'}, inplace=True)

# Unir os dois DataFrames
lucro_df = pd.merge(receita_agrupada, despesa_agrupada, on='Data', how='outer').fillna(0)

# Calcular o lucro
lucro_df['Lucro'] = lucro_df['Receita'] + lucro_df['Despesa']  # Lucro = Receita - Despesa (que é negativa)

# Obter o ano atual
ano_atual = datetime.now().year

# Criar a aplicação Dash
app = Dash(__name__)

# Layout da aplicação
app.layout = html.Div([
    dcc.Dropdown(
        id='ano-dropdown',
        options=[
            {'label': '2023', 'value': 2023},
            {'label': '2024', 'value': 2024}
        ],
        value=ano_atual,  # Valor padrão como ano atual
        clearable=False,
        style={'width': '50%'}
    ),
    dcc.Graph(id='grafico-anual')
])

@app.callback(
    Output('grafico-anual', 'figure'),
    Input('ano-dropdown', 'value')
)
def update_graph(selected_year):
    # Filtrar os dados para o ano selecionado
    filtered_data = lucro_df[lucro_df['Data'].dt.year == selected_year]

    # Criar uma lista de meses
    meses = pd.date_range(start=f'{selected_year}-01-01', end=f'{selected_year}-12-31', freq='M').strftime('%B').tolist()

    # Criar coluna de mês para reindexação
    filtered_data['Mês'] = filtered_data['Data'].dt.strftime('%B')

    # Reindexar os dados para garantir que todos os meses estejam presentes
    filtered_data = filtered_data.set_index('Mês').reindex(meses, fill_value=0).reset_index()

    # Calcular totais
    total_receita = filtered_data['Receita'].sum()
    total_despesa = filtered_data['Despesa'].sum()
    total_lucro = filtered_data['Lucro'].sum()

    # Criar o gráfico
    fig = make_subplots(rows=1, cols=1,
                        subplot_titles=(f'Análise Financeira de {selected_year}',))

    # Gráfico: Comparação Mensal de Receitas, Despesas e Lucro Total
    fig.add_trace(go.Bar(
        x=filtered_data['Mês'],
        y=filtered_data['Receita'],
        name='Receitas',
        marker_color='#007bff'
    ))

    fig.add_trace(go.Bar(
        x=filtered_data['Mês'],
        y=filtered_data['Despesa'],
        name='Despesas',
        marker_color='#ff7b00'
    ))

    fig.add_trace(go.Scatter(
        x=filtered_data['Mês'],
        y=filtered_data['Lucro'],
        mode='lines+markers+text',
        name='Lucro',
        marker_color='#00cc00',
        line_color='#00cc00',
        text=filtered_data['Lucro'].round(2),
        textposition='top center'
    ))

    # Adicionar totais na ponta das barras
    for i in range(len(filtered_data)):
        fig.add_annotation(
            x=filtered_data['Mês'][i],
            y=filtered_data['Receita'][i],
            text=f'R$ {filtered_data["Receita"][i]:.2f}',
            showarrow=False,
            font=dict(size=10),
            yshift=5  # Ajuste a posição vertical do texto
        )
        fig.add_annotation(
            x=filtered_data['Mês'][i],
            y=filtered_data['Despesa'][i],
            text=f'R$ {filtered_data["Despesa"][i]:.2f}',
            showarrow=False,
            font=dict(size=10),
            yshift=5  # Ajuste a posição vertical do texto
        )

    # Adicionar totais como caixas de Big Number, lado a lado
    fig.add_annotation(
        xref='paper', yref='paper',
        x=0.2, y=1.35,  # Centralizado na parte superior
        showarrow=False,
        text=f"<b>Receita:</b> R$ {total_receita:.2f}",
        font=dict(size=16, color='black'),
        bgcolor='rgba(255, 255, 255, 0.9)',
        bordercolor='black',
        borderwidth=2,
        borderpad=10,
        align='center'
    )

    fig.add_annotation(
        xref='paper', yref='paper',
        x=0.482, y=1.35,
        showarrow=False,
        text=f"<b>Despesa:</b> R$ {total_despesa:.2f}",
        font=dict(size=16, color='black'),
        bgcolor='rgba(255, 255, 255, 0.9)',
        bordercolor='black',
        borderwidth=2,
        borderpad=10,
        align='center'
    )

    fig.add_annotation(
        xref='paper', yref='paper',
        x=0.75, y=1.35,
        showarrow=False,
        text=f"<b>Lucro:</b> R$ {total_lucro:.2f}",
        font=dict(size=16, color='black'),
        bgcolor='rgba(255, 255, 255, 0.9)',
        bordercolor='black',
        borderwidth=2,
        borderpad=10,
        align='center'
    )

    # Personalizar o layout do gráfico
    fig.update_layout(
        xaxis_title='Mês',
        yaxis_title='Valor (R$)',
        font_family='Arial',
        font_color='#333333',
        title_font_size=20,
        xaxis_tickfont_size=12,
        yaxis_tickfont_size=12,
        bargap=0.1,
        plot_bgcolor='#f5f5f5',
        paper_bgcolor='#f5f5f5'
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [14]:
# Primeiro, vamos garantir que a coluna 'Data' esteja no formato de datetime
lucro_df['Data'] = pd.to_datetime(lucro_df['Data'])

# Criar novas colunas para ano e mês
lucro_df['Ano'] = lucro_df['Data'].dt.year
lucro_df['Mês'] = lucro_df['Data'].dt.month

# Filtrar os dados de 2023 para treino
train_df = lucro_df[lucro_df['Ano'] == 2023]

# Definir as variáveis independentes (X) e dependentes (y) para treino
X_train = train_df[['Mês', 'Receita', 'Despesa']]
y_train = train_df['Lucro']

# Criar e treinar o modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Previsões para 2024
ano_desejado = 2024
previsao_lucro = []

# Gerar previsões para cada mês de 2024
for mes in range(1, 13):
    # Aqui você pode definir uma lógica para prever a receita e a despesa
    # Para este exemplo, vamos usar a média histórica
    receita_prevista = train_df['Receita'].mean()  # Usar a média de 2023
    despesa_prevista = train_df['Despesa'].mean()  # Usar a média de 2023

    # Criar um DataFrame para a previsão
    X_previsao = pd.DataFrame({
        'Mês': [mes],
        'Receita': [receita_prevista],
        'Despesa': [despesa_prevista]
    })

    # Fazer a previsão
    lucro_previsto = model.predict(X_previsao)
    previsao_lucro.append(lucro_previsto[0])

# Exibir previsões
for mes, lucro in enumerate(previsao_lucro, start=1):
    print(f"Mês {mes}: Lucro Previsto = R$ {lucro:.2f}")


Mês 1: Lucro Previsto = R$ -2598.11
Mês 2: Lucro Previsto = R$ -1669.54
Mês 3: Lucro Previsto = R$ -1105.56
Mês 4: Lucro Previsto = R$ -647.15
Mês 5: Lucro Previsto = R$ -587.18
Mês 6: Lucro Previsto = R$ -533.99
Mês 7: Lucro Previsto = R$ -400.99
Mês 8: Lucro Previsto = R$ -192.56
Mês 9: Lucro Previsto = R$ -307.61
Mês 10: Lucro Previsto = R$ -941.19
Mês 11: Lucro Previsto = R$ -1072.77
Mês 12: Lucro Previsto = R$ -78.63
